https://assets.press.princeton.edu/chapters/s2_10291.pdf

(This is the One-Compartment Model of Repeated Doses, question 3 near end of above pdf)

In [ ]:
import reno as r

In [ ]:
t = r.TimeRef()
compartment = r.Model("one_compartment_model", steps=168)
with compartment:
    drug_in_system = r.Stock(doc="mass of medication in the (\"person's blood serum\"?)")
    ingested = r.Flow(doc="Pulsed inflow of medication when dosage is taken.")
    eliminated = r.Flow(doc="Rate of change of drug leaving the system.")
    
    absorption_fraction = r.Variable(.12)
    dosage = r.Variable(100 * 1000, doc="Dosage is 100 * 1000 micrograms")
    start = r.Variable(0, doc="Timestep of first dosage. (in hours)")
    interval = r.Variable(8, doc="Timesteps between each dosage. (in hours)")
    
    volume = r.Variable(3000, doc="Volume of blood serum, 3000 mL") 
    concentration = r.Variable(drug_in_system / volume)
    half_life = r.Variable(22, doc="Half-life of medication. (in hours)")
    elimination_constant = r.Variable(-r.log(0.5) / half_life)
    
    eliminated.eq = elimination_constant * drug_in_system
    ingested.eq = absorption_fraction * dosage * r.repeated_pulse(start, interval)

    ingested >> drug_in_system >> eliminated

In [ ]:
compartment.latex()

In [ ]:
ds = compartment()

In [ ]:
r.plot_trace_refs(compartment, [ds], [compartment.concentration])

In [ ]:
compartment.graph()

In [ ]:
compartment.graph(sparklines=True, sparkall=True)

Showing variance when run with probability distributions (varying dosage and absorption_fraction)

In [ ]:
ds2 = compartment(steps=168, n=1000, dosage=r.Uniform(90*1000, 110*1000), absorption_fraction=r.Normal(.11, .025))
ds2

In [ ]:
r.plot_trace_refs(compartment, [ds2], [compartment.concentration])

In [ ]:
# compartment.conc_100 = reno.Metric(compartment.concentration.timeseries[100])

Can we determine what dosage and absorption fraction would lead to a high concentration at a particular step?

In [ ]:
trace0 = compartment.pymc(
    n=4000, 
    dosage=r.Uniform(90*1000, 110*1000), 
    absorption_fraction=r.Normal(.11, .1),
    observations=[
        r.Observation(compartment.concentration.timeseries[100], 2, [40])
    ],
)

In [ ]:
r.plot_trace_refs(compartment, [trace0.prior, trace0.posterior], [compartment.dosage, compartment.absorption_fraction, compartment.concentration])

Example problem: we want to determine the absorption fraction for a particular drug, where dosage is controlled but possible variance in start/interval times
Below are values for some observations we can make, with the goal to highlight how more observations improves posterior distributions

In [ ]:
ds.concentration.values[0][100], ds.concentration.values[0][150], ds.concentration.values[0][30]

In [ ]:
# single observation case
trace1 = compartment.pymc(
    n=4000, 
    dosage=100*1000,
    start=r.DiscreteUniform(0, 1),
    interval=r.DiscreteUniform(7, 9),
    absorption_fraction=r.Normal(.15, .025),
    observations=[
        r.Observation(compartment.concentration.timeseries[100], 1, [15.5])
    ],
)


In [ ]:
r.plot_trace_refs(
    compartment, 
    {"prior": trace1.prior, "1 obs": trace1.posterior}, 
    [compartment.absorption_fraction, compartment.concentration],
    figsize=(10, 4)
)

In [ ]:
# two observed concentrations
trace2 = compartment.pymc(
    n=4000, 
    dosage=100*1000,
    start=r.DiscreteUniform(0, 1),
    interval=r.DiscreteUniform(7, 9),
    absorption_fraction=r.Normal(.15, .025),
    observations=[
        r.Observation(compartment.concentration.timeseries[100], 1, [15.5]),
        r.Observation(compartment.concentration.timeseries[150], 1, [15.0])
    ],
)

In [ ]:
r.plot_trace_refs(
    compartment, 
    {"prior": trace2.prior, "1 obs": trace1.posterior, "2 obs": trace2.posterior}, 
    [compartment.absorption_fraction, compartment.concentration],
    figsize=(10, 4)
)


In [ ]:
# three observations
trace3 = compartment.pymc(
    n=4000, 
    dosage=100*1000,
    start=r.DiscreteUniform(0, 1),
    interval=r.DiscreteUniform(7, 9),
    absorption_fraction=r.Normal(.15, .025),
    observations=[
        r.Observation(compartment.concentration.timeseries[100], 1, [15.5]),
        r.Observation(compartment.concentration.timeseries[150], 1, [15.0]),
        r.Observation(compartment.concentration.timeseries[30], 1, [9.5]),
    ],
)


In [ ]:
r.plot_trace_refs(
    compartment, 
    {"prior": trace2.prior, "1 obs": trace1.posterior, "2 obs": trace2.posterior, "3 obs": trace3.posterior},
    [compartment.absorption_fraction, compartment.concentration],
    figsize=(10, 4)
)

In [ ]:
print("Predicted mean:".rjust(15), trace3.posterior.absorption_fraction.values.mean())
print("Actual value:".rjust(15), compartment.absorption_fraction.eq.value)

In [ ]:
compartment.graph(
    sparklines=True,
    sparkall=True,
    sparkdensities=True,
    traces=[trace3.prior, trace3.posterior],
    exclude_vars=["half_life", "dosage", "elimination_constant", "volume", "concentration"]
)